# MARATÓ TV3: COVID 19

En gairebé 3 mesos, l'epidemia mundial del coronavirus portarà un any vigent. Tot i que cada vegada vegada estem més a prop de saber com afrontar el virus, hem vist algunes solucions tecnològiques al llarg d'aquest temps força interessants en les que la societat s'ha tingut d'adaptar, com per exemple, a l'ús diari aplicacions de delivering, així com aquelles pròpies del món mèdic.

Com a repte per aquesta Hackathon, he volgut veure i estudiar si el soroll de la tos d'un pacient és una possible indicació de si aquest pateix COVID o no.

Se sap que els sons respiratoris tenen propietats diagnòstiques. A dia d'avui, aquestes senyals les sol recullir el metge durant l'auscultació manual mitjançant un estetoscopi. 

En la investigació que he dut a terme he utilitzat sons corporals per analitzar i classificar els sons de la tos dels pacients amb COVID positiu o COVID negatiu. 

## DATA ANALYSIS

El conjunt de dades inicial utilitzat per construir les bases del meu model predictiu consisteix en 250 fitxers .mp3 i un full de càlcul .csv que detalla el seu identificador i els resultats de les proves del COVID.

Les dades consisteixen en un nombre significativament gran de pacients amb COVID negatiu, així com que hi ha un nombre més gran de dones participants i participants no fumadors. El conjunt de dades també inclou pacients d'entre 8 i 81 anys, amb una gran quantitat d'aquests d'entre 22 i 34 anys.

Quan parlem de so, generalment parlem d’una seqüència de vibracions amb diferents forces de pressió. Per tant, visualitzar el so significa visualitzar les ones d’alguna manera. Això ho podem aconseguir convertint els fitxers d’àudio en espectrogrames. 

Per convertir les dades en representacions d’espectrogrames, hem utilitzat la llibreria LibROSA, un paquet de codi obert de Python per a l’anàlisi de música i àudio. La representació resultant es pot mostrar com una imatge 2D i s’anomena Transformada de Fourier de Temps Curt. 

La llibreria LibROSA ens permet carregar un fitxer d’àudio i convertir-lo en un espectograma mitjançant el codi següent:

In [1]:
import librosa
import librosa.display
fname = 'test-1-audio001.wav'
samples, sample_rate = librosa.load(fname)

fig = plt.figure(figsize=[4,4])
ax = fig.add_subplot(111)
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)
ax.set_frame_on(False)
S = librosa.feature.melspectrogram(y=samples, sr=sample_rate)
librosa.display.specshow(librosa.power_to_db(S, ref=np.max))

ModuleNotFoundError: No module named 'librosa'

El preprocessament de conjunts de dades, l'extracció de funcions i l'enginyeria de funcions són els passos que es fan per extreure informació de les dades subjacents. En el context de l’aprenentatge automàtic, aquesta informació hauria de ser útil per predir la classe d’una mostra. En l'anàlisi d'àudio, aquest procés es basa en gran mesura en trobar components d'un senyal d'àudio que puguin ajudar a distingir-lo d'altres senyals.

Hem analitzat els sons de la tos mitjançant variables basades en la freqüència. Aquestes empremtes digitals capturen informació d'àudio universal i codifiquen matisos o patrons que no es poden fer fàcilment a mà. 

Algunes de les característiques extretes de les mostres de tos són:

1. Funcions basades en croma que calcula quant existeix de cada classe al senyal en to cromàtic (C, C♯, D, D♯, E, F, F♯, G, G♯, A, A♯, B).

2. Freqüència de Mel. Ens assenyala que qualsevol so generat per un ésser humà està determinat per la forma del seu tracte vocal (incloses la llengua, les dents, etc.). Si es pot determinar correctament aquesta forma, es pot representar amb precisió qualsevol so produït.

3. La mitjana quadràtica (RMS) amb forma d'ona que correspon a la intensitat del so.

## NEURAL NETWORK

Un cop hem extret amb èxit les funcions de les dades d’àudio subjacents, ajustem una xarxa neuronal senzilla amb un backend Keras i Tensorflow.

En el nostre cas, dividim les dades en un 70% d’entrenament i un 30% de proves. També escalem els valors de les funcions. És a dir, transformem les dades perquè s’adaptin a l’interval de 0 i 1.

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split 

# converteix les funcions i les etiquetes de classificació corresponents en numpy arrays

X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# codifica les etiquetes de classificació

le = LabelEncoder()
y = to_categorical(le.fit_transform(y))

# escala les característiqes

scaler = StandardScaler()
X = scaler.fit_transform(np.array(featuresdf.iloc[:, :-1], dtype = float))

# divideix el dataset 

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Un cop configurats els paràmetres, es pot definir i compilar la xarxa neuronal utilitzant alguna cosa similar al codi següent. Com que es tracta d’un problema de classificació binària, és a dir, intentem classificar si un so de la tos és COVID-positiu o COVID-negatiu, s’hauria d’indicar i definir l’entropia creuada binària.

In [ ]:
model = Sequential()
model.add(layers.Dense(26, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(18, activation='relu'))
model.add(layers.Dense(12, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Un cop el model és entrenat, s'avalua la precisió d'aquest. Basant-nos en el conjunt de dades inicial, els nostres resultats han demostrat que els sons de la tos poden predir el COVID-19 amb gairebé un 80% de precisió.

## CONCLUSIONS

Tot i que els resultats són un signe positiu, encara no són prou sòlids com per constituir una eina de selecció autònoma. 
Detectar lleugeres diferències en els sons de la tos amb precisió és una tasca realment difícil.

Per a poder millorar la precisió, seria recomenable poder tractar amb una quantitat de dades de major magnitud i millorar quelcom del codi. A més a més, com a part de la millora del model, el següent pas seria investigar si també es poden incorporar els espectrogrames dels sons de la tos. Els elements de classificacions sonores també es poden abordar com una tasca tradicional de classificació d’imatges mitjançant xarxes neuronals convolucionals que poden aconseguir una precisió més alta i efectiva.